Github code from authors stored in picrispr-main

TODO:

1. Preprocess data (dir is offtarget_260520_nuc.csv.zip (crisprsql dataset)).
we can use the code from load_data.py and encoding.py.
the author calls the functions from those classes in picrispr.py.

2. Run models (models.py)

3. Test model (test_input.csv)

In [ ]:
#Load files
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
 default_vals   nontransformed.png	      'picrispr data format.gsheet'   test_input.csv
 encoding.py    offtarget_260520_nuc.csv.zip   picrispr.py		      transformedback.png
 load_data.py  'output(1).gsheet'	       __pycache__		     'WillB (1).py'
 models         output.csv		       README.md		      WillB.py
 models.py      output_Option3RNN.csv	       requirements.txt


In [ ]:

!ls
#load data preproces here

 default_vals   nontransformed.png	      'picrispr data format.gsheet'   test_input.csv
 encoding.py    offtarget_260520_nuc.csv.zip   picrispr.py		      transformedback.png
 load_data.py  'output(1).gsheet'	       __pycache__		     'WillB (1).py'
 models         output.csv		       README.md		      WillB.py
 models.py      output_Option3RNN.csv	       requirements.txt


In [ ]:
import pandas as pd
filePath = "offtarget_260520_nuc.csv.zip"
df = pd.read_csv(filePath, low_memory=False)
print(df.columns)

Index(['id', 'experiment_id', 'target_chr', 'target_start', 'target_end',
       'target_strand', 'target_sequence', 'target_context', 'target_geneid',
       'grna_target_id', 'grna_target_chr', 'grna_target_start',
       'grna_target_end', 'grna_target_strand', 'grna_target_sequence',
       'genome', 'cell_line', 'measured', 'cleavage_freq', 'epigenetics_ids',
       'epigen_ctcf', 'epigen_dnase', 'epigen_rrbs', 'epigen_h3k4me3',
       'epigen_drip', 'energy_1', 'energy_2', 'energy_3', 'energy_4',
       'energy_5', 'GCContent', 'WSScore', 'YRScore', 'NucleotideBDM',
       'StrongWeakBDM', 'NuPoP_Occup_147_human', 'NuPoP_Viterbi_147_human',
       'NuPoP_Affinity_147_human', 'nuCpos_Occup_147_yeast',
       'nuCpos_Viterbi_147_yeast', 'nuCpos_Affinity_147_yeast',
       'VanDerHeijden', 'LeNupH3Q85C', 'mismatch_num',
       'energy_4*(energy_3/energy_2)', 'energy_2-energy_4*(energy_3/energy_2)',
       'energy_1-energy_4*(energy_3/energy_2)'],
      dtype='object')


In [ ]:

import sys
from encoding import oneHotSingleNuclTargetMismatchType, oneHotSingleNuclTargetMismatch, oneHotSingleNucl, normaliseCF
from models import mySequential, vecToMatEncoder, vecToMatEncoding
import pickle
from tqdm import tqdm
import xgboost as xgb
import torch
import tensorflow as tf
from scipy.stats import spearmanr
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from picrispr import CSVDataset


use_gpu = True

filenames = ["xgboost_interface_type_epi", "torch_eng", "torch_engnuc", "torch_interface_type_nuc", "tf_eng", "tf_engnuc"]
modelnames =["XGB_S3E2", "CNN_S2E0", "CNN_S4E0", "CNN_S5E2", "RNN_S2E3", "RNN_S4E3"]

# define models
seq_features = "target_sequence, grna_target_sequence, "
energy_features = "energy_2+energy_1-(energy_3*energy_4/energy_2), "
epigen_features = "epigen_ctcf, epigen_dnase, epigen_rrbs, epigen_h3k4me3, epigen_drip, "
nuc_features = "NucleotideBDM, NuPoP_Affinity_147_human, GCContent, "
numBpWise = 3

seq_energy_feat         = seq_features + energy_features + "experiment_id" #4 total
seq_energy_epi_feat     = seq_features + energy_features + epigen_features + "experiment_id" #9 total
seq_energy_nuc_feat     = seq_features + nuc_features + energy_features + "experiment_id"  #6 total
seq_energy_epi_nuc_feat = seq_features + nuc_features + energy_features + epigen_features + "experiment_id"  #12 total

# retrieve user input
filePath = "offtarget_260520_nuc.csv.zip"
modelNum   = 3
home       = "models"
regression = False
replace_all_nans = False

args = [("", home, "torch", False, regression, "s4", False), # E3+CRISPRNetStyle, CNN, s4
        ("", home, "torch", False, regression, "s2", False), # E3+CRISPRNetStyle, CNN, s2
        ("", home, "tf",    False, regression, "s4", False), # E3+CRISPRNetStyle, RNN, s4
        ("", home, "tf",    False, regression, "s2", False), # E3+CRISPRNetStyle, RNN, s2 THIS ONE
        ("", home, "torch", True,  regression, "s4", True),  # E2,                CNN, s4
        ("", home, "torch", True,  regression, "s2", True),  # E2,                CNN, s2
        ("", home, "tf",    True,  regression, "s4", True),  # E2,                RNN, s4
        ("", home, "tf",    True,  regression, "s2", True),  # E2,                RNN, s2
        ]

kwargs = [{'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True}, #yTHIS ONE
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False}
          ]

# load user dataset and chosen model
dataset = CSVDataset(filePath)

config = {"dbFields": kwargs[modelNum]['dbFields'],
          "numBpWise": kwargs[modelNum]['numBpWise'],
          "mode": args[modelNum][2],
          "mismatchType": args[modelNum][3],
          "interfaceMode": args[modelNum][6],
          "chooseSpecies": ["hg19", "hg38"],
          "regression": args[modelNum][4],
          "CRISPRNetStyle": kwargs[modelNum]['CRISPRNetStyle'],
          }

print(config)

filenameAppendix  = "_interface" if config["interfaceMode"] else ""
filenameAppendix += "_type"      if config["mismatchType"] and config["interfaceMode"]  else ""
filenameAppendix += "_"+args[modelNum][5]

print(config["mode"], filenameAppendix, "__________________________________")

if config["mismatchType"]:
    oneHotFct = oneHotSingleNuclTargetMismatchType
    featurenames = ['A_match', 'T_match', 'C_match', 'G_match',
                    'A_mismT', 'T_mismC', 'C_mismG', 'G_mismA',
                    'A_mismC', 'T_mismG', 'C_mismA', 'G_mismT',
                    'A_mismG', 'T_mismA', 'C_mismT', 'G_mismC']
elif config["interfaceMode"]:
    oneHotFct = oneHotSingleNuclTargetMismatch
    featurenames = ['A', 'A_mism', 'T', 'T_mism', 'C', 'C_mism', 'G', 'G_mism']
else:
    oneHotFct = oneHotSingleNucl
    featurenames = ['A',           'T',           'C',           'G']

featurenames.extend(" ".join(config["dbFields"].split()).split(', ')[2:]) # append whatever database fields apart from guide and target sequence are used


# get data matrix from user input
filenameAppendix += "_class" if not config["regression"] else ""

print("encoding dataset...", end = '')
dm = dataset.getDataMatrix(config["dbFields"], oneHotFct, normaliseCF, chooseSpecies=config["chooseSpecies"], filenameAppendix=filenameAppendix,
                            featurenames=featurenames, mode=config["mode"], numBpWise=config["numBpWise"], test_size=0.2, doSplit=True,
                            CRISPRNetStyle=config["CRISPRNetStyle"], replace_all_nans=replace_all_nans)
print("done")

print("config[mode] is ", config["mode"], " so dm.mode was set to be ", dm.mode)
dm.mode = config["mode"]
dm.interfaceMode = config["interfaceMode"]
dm.mismatchType = config["mismatchType"]
dm.regression = config["regression"]
dm.CRISPRNetStyle = config["CRISPRNetStyle"]

# load model
print("loading model...", end = '')
#result = TrainResult.load(home, config["mode"], filenameAppendix, device="gpu:0" if use_gpu else "cpu")
print("done")

# predict on ext set
isHPC = True
#bs = int(7e4) if isHPC or dm.mode != "torch" else 35000
bs = 1024
# don't use experiment_id column
dm.toDense()
dm = dm.dropColumn(-1)

print("preparing dataset...", end = '')
trainDataset, validDataset, extDataset = dm.prepareDataset(cutoff_class=-4, addGaussian=False)
print("dataset dim: ", len(list(trainDataset)))
#trainDataset = PaddedDataset(trainDataset, desired_length)
#validDataset = PaddedDataset(validDataset, desired_length)
#extDataset = PaddedDataset(extDataset, desired_length)
#dm.prepareDataloaders(trainDataset, validDataset, extDataset, bs, balanceClasses=False, ignoreExtSet=True, doSampling=False)
#print("done")

{'dbFields': 'target_sequence, grna_target_sequence, energy_2+energy_1-(energy_3*energy_4/energy_2), experiment_id', 'numBpWise': 0, 'mode': 'tf', 'mismatchType': False, 'interfaceMode': False, 'chooseSpecies': ['hg19', 'hg38'], 'regression': False, 'CRISPRNetStyle': True}
tf _s2 __________________________________
encoding dataset...

103it [00:00, 10546.94it/s]
1505it [00:00, 14966.16it/s]
78it [00:00, 14910.70it/s]
3801it [00:00, 17163.94it/s]
51it [00:00, 11185.98it/s]
5413it [00:00, 15772.46it/s]
549it [00:00, 14617.64it/s]
44624it [00:02, 16080.94it/s]
72it [00:00, 14794.00it/s]
29327it [00:01, 17751.09it/s]
7374it [00:00, 26172.92it/s]
41429it [00:01, 20770.74it/s]
866it [00:00, 18518.46it/s]
6386it [00:00, 18709.80it/s]
231it [00:00, 17048.52it/s]
27995it [00:01, 19746.22it/s]
11it [00:00, 12476.30it/s]
2380it [00:00, 26303.51it/s]
86it [00:00, 17900.36it/s]
2438it [00:00, 14988.18it/s]
108it [00:00, 14590.29it/s]
3258it [00:00, 18852.62it/s]
61it [00:00, 16712.56it/s]
1747it [00:00, 15977.94it/s]
272it [00:00, 17291.12it/s]
23463it [00:01, 20737.80it/s]
203it [00:00, 23982.30it/s]
12373it [00:00, 19709.29it/s]
23it [00:00, 10451.68it/s]
21519it [00:01, 20390.80it/s]
11213it [00:00, 14802.36it/s]
2895it [00:00, 15496.49it/s]
/content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0

using GPU device Tesla T4
done
config[mode] is  tf  so dm.mode was set to be  tf
loading model...done
preparing dataset...dataset dim:  201482


In [ ]:
l0 = []
l1 = []

for batch in trainDataset:
    input, target, extra = batch
    if target.item() == 0:
      l0.append((input, target))
    else:
      l1.append((input, target))

1-0.9155954378058586

0.08440456219414139

In [ ]:
print(len(l0))
print(len(l1))

184478
17004


In [ ]:
import numpy as np

l0 = np.array(l0)
l1 = np.array(l1)
np.random.shuffle(l0)

l0 = l0[:len(l1)]

l = np.concatenate((l0, l1))

np.random.shuffle(l)

<ipython-input-8-6ce3c8c4995c>:3: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  l0 = np.array(l0)
<ipython-input-8-6ce3c8c4995c>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  l0 = np.array(l0)
<ipython-input-8-6ce3c8c4995c>:4: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_i

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, idx):
        input_sample = torch.tensor(self.inputs[idx])
        target_sample = torch.tensor(self.targets[idx])

        sample = (input_sample, target_sample)

        return sample

In [ ]:
trainDataset = MyDataset(l[:,0], l[:,1])

In [ ]:
bs =

In [ ]:
from torch.utils.data import DataLoader

traindataloader = DataLoader(trainDataset, batch_size=bs, shuffle=True)
testdataloader = DataLoader(validDataset, batch_size=bs, shuffle=True)

TwoLayerNet: 91.56% Accuracy
ThreeLayerNet: Epoch [15/15], Train Accuracy: 0.9622
Epoch [15/15], Test Accuracy: 0.9321
ConvNet

In [ ]:
from models import TwoLayerNet, ThreeLayerNet, LinearRegressionModel, ConvolutionalNet, createDeepCRISPRmodel
import torch.nn as nn
import torch.optim as optim
import numpy as np

dim = 512
heads = 8
layers = 6
num_epochs = 100

#ADDING NUC GOES FROM 4 TO 6
lossf = nn.BCELoss() #nn.CrossEntropyLoss()
# model = ConvolutionalNet(False)
# model = ThreeLayerNet(139, 256, 1)
#model = TwoLayerNet(139, 256, 1)
#model = LinearRegressionModel(139, 256) RuntimeError: shape '[1, 60000]' is invalid for input of size 15360000

#do conv, threelayer, GaussianNoise, NormaliseCustom, XGBoostTree, CRNNCrisprModel, ConjoinedConvolutionalNet and other stuff in models.py
#only classification
# lossf = nn.MSELoss() for regression
model = ConvolutionalNet(False)
# model = ThreeLayerNet(139, 256, 1)
# model = createDeepCRISPRmodel(regression = False, device = True, seqDim=4, epiStart=4, epiDim=10, lossfct='bce', optimiser='adam')
#LOOK HERE FOR CHANGING FEATURES
# (self, regression, p=0.0, device=None, seqDim=4, epiStart=4, epiDim=10, mean = 0, stdev = 0.254, batchnorm_momentum = 0.1):
# model = TwoLayerNet(139, 256, 1) [91.56%]
#model = LinearRegressionModel(139, 256) RuntimeError: shape '[1, 60000]' is invalid for input of size 15360000

encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=layers)

model = nn.Sequential(
    nn.Linear(139,dim),
    transformer_encoder,
    nn.Linear(dim, 1),
    nn.Sigmoid()
)

optimizer = optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(num_epochs):

    model.train()

    total_correct = 0
    total_samples = 0

    for batch in traindataloader:
        inputs, targets = batch
        # Forward pass

        outputs = model(inputs).squeeze()

        pred = outputs.detach().numpy()
        labels = targets.numpy()

        # Assuming labels are in the range [0, 1] (0 for class 0, 1 for class 1)
        #labels = labels.float().view(-1, 1)  # Ensure labels are of type float and have shape (batch_size, 1)

        # Compute accuracy
        predictions = pred > 0.5 #pred[:,1] > 0.5  # Threshold at 0.5 for binary classification

        correct = np.sum(predictions == labels.astype(int))
        total_correct += correct
        total_samples += len(labels)

        # Compute loss
        loss = lossf(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {total_correct/total_samples:.4f}')

    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0


    with torch.no_grad():  # Disable gradient computation during validation
        for batch in testdataloader:
            inputs, targets, extra = batch
            outputs = model(inputs).squeeze()

            #outputs = F.softmax(outputs, dim=1)

            pred = outputs.numpy()
            labels = targets.numpy()

            # Assuming labels are in the range [0, 1] (0 for class 0, 1 for class 1)
            #labels = labels.float().view(-1, 1)  # Ensure labels are of type float and have shape (batch_size, 1)

            # Compute accuracy
            predictions = pred > 0.5 #pred[:,1] > 0.5  # Threshold at 0.5 for binary classification

            correct = np.sum(predictions == labels.astype(int))
            total_correct += correct
            total_samples += len(labels)


    #accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {total_correct/total_samples:.4f}')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
<ipython-input-9-e968f9abd1eb>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_sample = torch.tensor(self.inputs[idx])
<ipython-input-9-e968f9abd1eb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_sample = torch.tensor(self.targets[idx])


Epoch [1/100], Train Accuracy: 0.8100
Epoch [1/100], Test Accuracy: 0.8575
Epoch [2/100], Train Accuracy: 0.8901
Epoch [2/100], Test Accuracy: 0.8413
Epoch [3/100], Train Accuracy: 0.9052
Epoch [3/100], Test Accuracy: 0.8537
Epoch [4/100], Train Accuracy: 0.9139
Epoch [4/100], Test Accuracy: 0.8235
Epoch [5/100], Train Accuracy: 0.9160
Epoch [5/100], Test Accuracy: 0.8039
Epoch [6/100], Train Accuracy: 0.9216
Epoch [6/100], Test Accuracy: 0.8039
Epoch [7/100], Train Accuracy: 0.9244
Epoch [7/100], Test Accuracy: 0.8461
Epoch [8/100], Train Accuracy: 0.9276
Epoch [8/100], Test Accuracy: 0.8801
Epoch [9/100], Train Accuracy: 0.9315
Epoch [9/100], Test Accuracy: 0.8902
Epoch [10/100], Train Accuracy: 0.9318
Epoch [10/100], Test Accuracy: 0.9044
Epoch [11/100], Train Accuracy: 0.9335
Epoch [11/100], Test Accuracy: 0.8684
Epoch [12/100], Train Accuracy: 0.9380
Epoch [12/100], Test Accuracy: 0.8641
Epoch [13/100], Train Accuracy: 0.9377
Epoch [13/100], Test Accuracy: 0.8644
Epoch [14/100], T

KeyboardInterrupt: ignored